# CNN

In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.

In [2]:
class Model(nn.Module):
    # Input Layer (4 features of flowers) --> HL1 (number of neurons) --> HL2(n) --> Ouput(3 Classes of Iris Flower)
    # fc -- fully connected 1 , fully connected 2 
    def __init__(self, in_features=4, h1=8, h2=9, out_features=3):
        super().__init__()
        self.fc1 = nn.Linear(in_features,h1)    # start from in_features and move to h1 , fc(fully connected)
        self.fc2 = nn.Linear(h1,h2)             # start from h1 and move to h2 
        self.out = nn.Linear(h2,out_features)   # start from h2 and move to out_features 
        
                                                # Relu stands for rectified linear unit
    def forward(self,x):
        x = F.relu(self.fc1(x))                 # if output is less than 0 , then use 0 , else leave what it is. 
        x = F.relu(self.fc2(x))                 # if output is less than 0 , then use 0 , else leave what it is. 
        x = self.out(x)
        return x 

In [4]:
# Pick a manual seed for randomization 
torch.manual_seed(41)
# Create instance of a model
model = Model()

In [5]:
import pandas as pd 
import matplotlib.pyplot as plt 
%matplotlib inline

In [6]:
url = 'https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv'
my_df = pd.read_csv(url)
my_df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
